# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [65]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [66]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [67]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [68]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [69]:
def foldl2(f, x0, lst): 
    return foldr(lambda x, z: lambda y: z(f(y, x)), lambda j: j, lst)(x0)

In [70]:
def foldr2(f, x0, lst): 
    return foldl(lambda z, x: lambda y: z(f(x, y)), lambda j: j, lst)(x0)

In [71]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [72]:
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [73]:
from collections import Counter


def check_inv(a, b):
    if len(b) > len(a):
        return False
    
    return Counter(b) - Counter(a) & Counter(b) == {}

In [74]:
assert check_inv('abcrotm', 'tro')

In [75]:
assert not check_inv('abcdef', 'ghj')

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [76]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
        if not self.left and not self.right:
            yield self.value

    def __str__(self):
        return 'Tree [' + str(self.value) + ']'

    def __repr__(self):
        return 'Tree [' + 'value: ' + str(self.value) + '; left: ' + str(
            self.left) + '; right: ' + str(self.right) + ']'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [77]:
tree

Tree [value: 0; left: Tree [1]; right: Tree [2]]

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [78]:
import re


def compare_operators(stack, x):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    try:
        return precedence[x] <= precedence[stack[-1]]
    except KeyError:
        return False


def infix_to_postfix(inputs):
    outputs = []
    stack = []
    for x in inputs:
        if x.isdigit():
            outputs.append(int(x))
        elif x == '(':
            stack.append(x)
        elif x == ')':
            while stack and stack[-1] != '(':
                outputs.append(stack.pop())
            stack.pop()
        else:
            while stack and compare_operators(stack, x):
                outputs.append(stack.pop())
            stack.append(x)
    while stack:
        outputs.append(stack.pop())
    return outputs


def calc(expr):
    infix = re.findall(r'[-+/*()]|\d+(?:\.\d+)?', expr)
    postfix = infix_to_postfix(infix)

    stack = []
    for x in postfix:
        if type(x) is int:
            stack.append(x)
        else:
            operand2, operand1 = stack.pop(), stack.pop()

            if x == '+':
                stack.append(operand1 + operand2)
            elif x == '-':
                stack.append(operand1 - operand2)
            elif x == '*':
                stack.append(operand1 * operand2)
            elif x == '/':
                stack.append(operand1 // operand2)

    return stack.pop()

In [79]:
calc('2 * (15 - 3 * 4) - 2') == 4

True

In [80]:
calc('8 * 7 - 5 * (10 - 4) - 18 / (2 + 1)') == 20

True

In [81]:
calc('10 / (5 * (3 - 1))') == 1

True